In [7]:
with open("mitchish7.csv", "r", encoding="UTF8") as f:
    lines = iter(f)
    columns = next(lines).rstrip("\n").split("\t")
    data = []
    for line in lines:
        line = line.rstrip("\n").split("\t")
        d = {}
        for column_name, v in zip(columns, line):
            if len(v) > 0:
                d[column_name] = float(v)
        data.append(d)

In [8]:
len(data)

145742

In [25]:
embedding_norm_column = 'optim/grad/transformer.wte.weight.norm'
head_norm_column = 'optim/grad/transformer.ff_out.weight.norm'
grad_norm_columns = [k for k in columns if k.startswith("optim/grad") and k.endswith(".norm")]
grad_norm_columns.remove(embedding_norm_column)
grad_norm_columns.remove(head_norm_column)

layer = 0
layer_to_norm_columns = {}
while True:
    cs = []
    for c in grad_norm_columns:
        if f".{layer}." in c:
            cs.append(c)
    if len(cs) <= 0:
        break
    for c in cs:
        grad_norm_columns.remove(c)
    layer_to_norm_columns[layer] = cs
    layer += 1

# Make sure all layers have the same number of metrics
assert(1 == len(set(len(cs) for cs in layer_to_norm_columns.values())))

layer_to_norm_columns

{0: ['optim/grad/transformer.blocks.0.att_proj.weight.norm',
  'optim/grad/transformer.blocks.0.attn_out.weight.norm',
  'optim/grad/transformer.blocks.0.ff_out.weight.norm',
  'optim/grad/transformer.blocks.0.ff_proj.weight.norm'],
 1: ['optim/grad/transformer.blocks.1.att_proj.weight.norm',
  'optim/grad/transformer.blocks.1.attn_out.weight.norm',
  'optim/grad/transformer.blocks.1.ff_out.weight.norm',
  'optim/grad/transformer.blocks.1.ff_proj.weight.norm'],
 2: ['optim/grad/transformer.blocks.2.att_proj.weight.norm',
  'optim/grad/transformer.blocks.2.attn_out.weight.norm',
  'optim/grad/transformer.blocks.2.ff_out.weight.norm',
  'optim/grad/transformer.blocks.2.ff_proj.weight.norm'],
 3: ['optim/grad/transformer.blocks.3.att_proj.weight.norm',
  'optim/grad/transformer.blocks.3.attn_out.weight.norm',
  'optim/grad/transformer.blocks.3.ff_out.weight.norm',
  'optim/grad/transformer.blocks.3.ff_proj.weight.norm'],
 4: ['optim/grad/transformer.blocks.4.att_proj.weight.norm',
  'opti

In [24]:
columns_we_need = {embedding_norm_column, head_norm_column}
for layer_columns in layer_to_norm_columns.values():
    columns_we_need |= set(layer_columns)

data = [
    d
    for d in data
    if len(d.keys() & columns_we_need) == len(columns_we_need)
]

len(data)

14573

In [51]:
import numpy as np

step_counts = []
grad_norms_per_layer = np.zeros((len(data), len(layer_to_norm_columns) + 2), np.float64)
for i, d in enumerate(data):
    step_counts.append(int(d["_step"]))
    grad_norms_per_layer[i, 0] = d[embedding_norm_column]
    for layer_index in range(len(layer_to_norm_columns)):
        norm_columns = layer_to_norm_columns[layer_index]
        value = sum(d[c] for c in norm_columns) / len(norm_columns)
        grad_norms_per_layer[i, layer_index + 1] = value
    grad_norms_per_layer[i, -1] = d[head_norm_column]

assert len(grad_norms_per_layer) == len(step_counts)
(grad_norms_per_layer.min(), grad_norms_per_layer.max(), grad_norms_per_layer.mean(), grad_norms_per_layer.std())

(0.011589738307520747, 439.9797668457031, 0.03332919221355251, 0.8950047969909)

In [54]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LogNorm

dpi = 96
fig, ax = plt.subplots()
height, width = grad_norms_per_layer.shape
fig.set_figwidth(width)
fig.set_figheight(min(10 * height, 2**15) / dpi)
ax.set_yticklabels(step_counts)
c = ax.pcolor(grad_norms_per_layer, norm=LogNorm(vmin=grad_norms_per_layer.min(), vmax=4))
fig.tight_layout()
plt.show()

/var/folders/cq/y9vz0k454c59ytl7h9xhcfzw0000gp/T/ipykernel_63441/4190084774.py:10: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(step_counts)
